In [1]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/imagenet_classes.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/ILSVRC2012_validation_ground_truth.txt
!rm data/ILSVRC2012_img_val.tar

--2022-05-04 16:02:17--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  2.15MB/s    in 18m 31s 

2022-05-04 16:20:49 (5.79 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-05-04 16:21:23--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt’

map_clsloc.txt      100%[====

In [2]:
import tensorflow as tf
import numpy as np

model = tf.keras.applications.EfficientNetB6(weights='imagenet')

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')

validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
for i in range(len(mapping_list_from)):
  mapping_list_to.append(mapping_list_from[i].split(" ")[1])
  mapping_list_from[i] = mapping_list_from[i].split(" ")[0]

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

174473216/174460376 [==============================] - 3s 0us/step


In [3]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  img = tf.keras.preprocessing.image.load_img(img_path, target_size=(528, 528))
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = tf.keras.applications.efficientnet.preprocess_input(x)
  preds = model.predict(x)

  id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=1)[0][0][0]
  index = mapping_list_from.index(id)

  if mapping_list_to[index] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=5)[0][ii][0]
      index = mapping_list_from.index(id)
      if mapping_list_to[index] == validation_list[i]:
        top_5 = top_5 + 1
        break

  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))

accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

49152/35363 [=========================================] - 0s 0us/step
1%, Top 1:0.8 Top 5:0.974
2%, Top 1:0.809 Top 5:0.974
3%, Top 1:0.8233333333333334 Top 5:0.9706666666666667
4%, Top 1:0.817 Top 5:0.97
5%, Top 1:0.8196 Top 5:0.9704
6%, Top 1:0.822 Top 5:0.9686666666666667
7%, Top 1:0.82 Top 5:0.9691428571428572
8%, Top 1:0.82175 Top 5:0.96775
9%, Top 1:0.8228888888888889 Top 5:0.9668888888888889
10%, Top 1:0.8234 Top 5:0.9666
11%, Top 1:0.8250909090909091 Top 5:0.9647272727272728
12%, Top 1:0.8255 Top 5:0.9665
13%, Top 1:0.8276923076923077 Top 5:0.9663076923076923
14%, Top 1:0.8261428571428572 Top 5:0.9661428571428572
15%, Top 1:0.8270666666666666 Top 5:0.9657333333333333
16%, Top 1:0.82775 Top 5:0.965375
17%, Top 1:0.8292941176470588 Top 5:0.9654117647058823
18%, Top 1:0.8274444444444444 Top 5:0.9653333333333334
19%, Top 1:0.8285263157894737 Top 5:0.9653684210526315
20%, Top 1:0.8283 Top 5:0.9649
21%, Top 1:0.8277142857142857 Top 5:0.965047619047619
22%, Top 1:0.8267272727272728 To